In [1]:
from fomo.pipelines.train import Learner
from fomo.pipelines.types.learner_args import LearnerArgs
import torch

/home/jupyter/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
learner_args = LearnerArgs()
learner_args.device = "cuda"
learner_args.epochs = 200
learner_args.model_type = "clip_transformer_adapter_text"
learner_args.train_eval_size = [816, 816]
learner_args.text_prompt_template = "a photo of a {}, a type of flower"
#learner_args.text_prompt_template = "a photo of a {}."
learner_args.print_freq = 200
learner_args.save_freq = 200
learner_args.learning_rate = 5e-3
learner_args.use_wandb = True
learner_args.patience = 10
learner_args.batch_size = 256
learner_args.momentum = 0.5
learner_args.train_subsample = "base"
learner_args.dataset = "oxford_flowers"
#learner_args.dataset = "cifar10"


learner = Learner(learner_args)

Turning off gradients in both the image and the text encoder
Parameters to be updated: {'adapter.mha.in_proj_bias', 'adapter.downsampler.1.weight', 'adapter.mha.out_proj.bias', 'image_adapter.fc.0.weight', 'adapter.downsampler.0.weight', 'adapter.upsampler.2.weight', 'adapter.upsampler.0.bias', 'adapter.upsampler.0.weight', 'adapter.upsampler.2.bias', 'image_adapter.fc.2.weight', 'adapter.downsampler.1.bias', 'adapter.mha.in_proj_weight', 'adapter.downsampler.0.bias', 'adapter.mha.out_proj.weight'}
Number of learnable paramms: 238784


In [ ]:
learner.run()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dqmiss. Use `wandb login --relogin` to force relogin


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch: [0][0/4]	Time  4.632 ( 4.632)	Data  3.237 ( 3.237)	Loss 1.1008e+00 (1.1008e+00)	Acc@1  74.61 ( 74.61)


 25%|██▌       | 1/4 [00:03<00:11,  3.81s/it]

Validate: [0/4]	Time  3.810 ( 3.810)	Loss 1.0277e+00 (1.0277e+00)	Prompt Acc@1  71.88 ( 71.88)


...

Epoch: [8][0/4]	Time  3.837 ( 3.837)	Data  2.918 ( 2.918)	Loss 9.4440e-01 (9.4440e-01)	Acc@1  71.88 ( 71.88)


 25%|██▌       | 1/4 [00:03<00:10,  3.59s/it]

Validate: [0/4]	Time  3.588 ( 3.588)	Loss 9.6506e-01 (9.6506e-01)	Prompt Acc@1  75.00 ( 75.00)


100%|██████████| 4/4 [00:05<00:00,  1.41s/it]


 * Prompt Acc@1 74.877
